In [12]:
import pandas as pd

rides = pd.read_parquet('../data/transformed/validated_rides_2024_01.parquet')

rides

,pickup_datetime,pickup_location_id
0,2024-01-01 00:57:55,186
1,2024-01-01 00:03:00,140
2,2024-01-01 00:17:06,236
3,2024-01-01 00:36:38,79
4,2024-01-01 00:46:51,211
...,...,...
2964619,2024-01-31 23:45:59,107
2964620,2024-01-31 23:13:07,114
2964621,2024-01-31 23:19:00,211
2964622,2024-01-31 23:07:23,107


In [13]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('h')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2024-01-01 00:57:55,186,2024-01-01 00:00:00
1,2024-01-01 00:03:00,140,2024-01-01 00:00:00
2,2024-01-01 00:17:06,236,2024-01-01 00:00:00
3,2024-01-01 00:36:38,79,2024-01-01 00:00:00
4,2024-01-01 00:46:51,211,2024-01-01 00:00:00
...,...,...,...
2964619,2024-01-31 23:45:59,107,2024-01-31 23:00:00
2964620,2024-01-31 23:13:07,114,2024-01-31 23:00:00
2964621,2024-01-31 23:19:00,211,2024-01-31 23:00:00
2964622,2024-01-31 23:07:23,107,2024-01-31 23:00:00


floor function rounds the date to previous hour, i.e., for hours from `2024-01-01 00:00:00` to `2024-01-01 23:59:59` will be rounded down to `2024-01-01 00:00:00`

Now we will group by to get per hour rides

In [14]:
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides

,pickup_hour,pickup_location_id,0
0,2024-01-01 00:00:00,4,25
1,2024-01-01 00:00:00,7,4
2,2024-01-01 00:00:00,9,1
3,2024-01-01 00:00:00,10,6
4,2024-01-01 00:00:00,12,4
...,...,...,...
77525,2024-01-31 23:00:00,260,2
77526,2024-01-31 23:00:00,261,12
77527,2024-01-31 23:00:00,262,9
77528,2024-01-31 23:00:00,263,53


column name needs to be changed

In [15]:
agg_rides.rename(columns={
    0: 'rides'
}, inplace=True)
agg_rides

,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,4,25
1,2024-01-01 00:00:00,7,4
2,2024-01-01 00:00:00,9,1
3,2024-01-01 00:00:00,10,6
4,2024-01-01 00:00:00,12,4
...,...,...,...
77525,2024-01-31 23:00:00,260,2
77526,2024-01-31 23:00:00,261,12
77527,2024-01-31 23:00:00,262,9
77528,2024-01-31 23:00:00,263,53


In [16]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DateOffset) -> pd.DataFrame:
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='h'
    )
    output = pd.DataFrame()
    
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]

        # quick way to add missing dates with 0 in a series
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i])

    output = output.reset_index().rename(columns={'index': 'pickup_hour'})

    return output

In [17]:
agg_rides_all_slots = add_missing_slots(agg_rides)

100%|██████████| 260/260 [00:00<00:00, 1053.26it/s]


In [18]:
agg_rides_all_slots

,pickup_hour,rides,pickup_location_id
0,2024-01-01 00:00:00,25,4
1,2024-01-01 01:00:00,29,4
2,2024-01-01 02:00:00,34,4
3,2024-01-01 03:00:00,31,4
4,2024-01-01 04:00:00,32,4
...,...,...,...
193435,2024-01-31 19:00:00,0,245
193436,2024-01-31 20:00:00,0,245
193437,2024-01-31 21:00:00,0,245
193438,2024-01-31 22:00:00,0,245


### Plotting timeseries data

In [20]:
from typing import Optional, List
import plotly.express as px

def plot_rides(
        rides: pd.DataFrame,
        locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none",
    )

    fig.show()

In [21]:
plot_rides(agg_rides_all_slots, locations=[43])

In [22]:
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2024_01.parquet')